<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/L2_Exercicio7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**7) Use o Apriori algoritmo para encontrar as regras de associação do conjunto de transações no arquivo transacoes.txt. Considere que os valores iguais a 1 indique a presença do item na transação. Para referência, a primeira coluna é x1, a segunda é x2 e assim em diante. Define os valores de confiança mínima igual a 0,85 e suporte mínimo de 0,5 e encontre as regras de associação. Apresente as regras encontradas.**

In [ ]:
import pandas as pd
import numpy as np
import itertools

conf_min = 0.85
sup_min = 0.5

colunas = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 
           'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 
           'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21']

transacoes = pd.read_csv("transacoes.txt", sep=" ", header=None, names=colunas)

In [ ]:
def regras_apriori(data, support_min):

  support = {}
  L = set(data.columns)
  
  for i in range(1, len(L)+1):
    c = list(itertools.combinations(L, i))

    L = set()

    for j in list(c):
      sup = data.loc[:,j].product(axis=1).sum()/len(data.index)
      if sup >= support_min:
        support[j] = sup
        L = set(set(L) | set(j))
  
  result = pd.DataFrame(list(support.items()), columns=["Itens", "Suporte"])

  return result

In [ ]:
def regras_associacao(data, confidence_min):
  support = pd.Series(data.Suporte.values, index=data.Itens).to_dict()
  L = data.Itens.values
  p = list(itertools.permutations(L, 2))

  datas = []
  
  for i in p:
    if set(i[0]).issubset(i[1]):
      
      conf = support[i[1]]/support[i[0]]

      if conf >= confidence_min:
        j = list(i[1])
        for k in list(i[0]):
          j.remove(k)
        j = tuple(j)
        datas.append([i[0], j, support[i[1]], conf])

  result = pd.DataFrame(datas, columns=["Antecedente", 
                                        "Consequente", 
                                        "Suporte", 
                                        "Confiança"])
  
  return result

In [ ]:
#Verifica regras que atendem a condição de suporte mínimo
r = regras_apriori(transacoes, sup_min)
r

,Itens,Suporte
0,"(x3,)",1.000000
1,"(x10,)",0.833333
2,"(x20,)",1.000000
3,"(x10, x20)",0.833333
4,"(x10, x3)",0.833333
5,"(x20, x3)",1.000000
6,"(x10, x3, x20)",0.833333


In [ ]:
#Verifica regras de associação que atendem a necessidade de confiança mínima
regras_associacao(r, conf_min)

,Antecedente,Consequente,Suporte,Confiança
0,"(x3,)","(x20,)",1.000000,1.0
1,"(x10,)","(x20,)",0.833333,1.0
2,"(x10,)","(x3,)",0.833333,1.0
3,"(x10,)","(x3, x20)",0.833333,1.0
4,"(x20,)","(x3,)",1.000000,1.0
5,"(x10, x20)","(x3,)",0.833333,1.0
6,"(x10, x3)","(x20,)",0.833333,1.0


In [ ]:
##Código de teste
#from mlxtend.frequent_patterns import apriori, association_rules

#frq_items = apriori(transacoes, min_support=sup_min, use_colnames=True)

## Collecting the inferred rules in a dataframe
#rules = association_rules(frq_items, metric ="confidence", min_threshold = conf_min)
#rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
#rules